# DEMO

In [ ]:
from qutip import *
import numpy as np 
import matplotlib.pyplot as plt
import math
from skimage.feature import peak_local_max
import pickle
import os
import glob
import heapq

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

saveDir = 'peaks'

##  plot peaks

In [ ]:
def routine(E_i,wd_i):
    
    print('running param set {} {}'.format(E_i, wd_i))
    
    # Define all the variables 
    kappa = 0.0012 # 0.0012 
    gJC = 0.3347 # 0.3347
    wc = 10.5665 # Cavity frequency/detuning 10.5665
    w0 = 8.1831 # Qubit frequency 8.1831
    gamma=0.0001 # 0.0001
    Emax =0.01 # 0.01
    EN=1
    #E = 0.01 #0.01(original) # Epsilon
    E = E_i
    N = 50 #50
    nloop = 1

    #wd = 10.6005 # Driving frequency (10.6005 original)
    wd = wd_i
    wlist = np.linspace(wd, wd,nloop)

    # Identity operators are defined for the space of the light field and the space of the atom

    ida = identity(N)
    idatom = identity(2)

    # Define cavity field and atomic operators

    a  = tensor(destroy(N),idatom)
    sm = tensor(ida,sigmam())

    # Hamiltonian # Reminder check hamiltonian from caltech paper

    H1= 1*gJC*(a.dag()*sm + sm.dag()*a) + 1*E*(a+a.dag())

    # Collapse Operators

    C1    = math.sqrt(kappa)*a
    C2    = math.sqrt(gamma)*sm

    C1dC1 = C1.dag()*C1
    C2dC2 = C2.dag()*C2

    # Calculate the Liouvillian

    L1 = spre(C1)*spost(C1.dag())-0.5*spre(C1dC1)-0.5*spost(C1dC1)
    L2 = spre(C2)*spost(C2.dag())-0.5*spre(C2dC2)-0.5*spost(C2dC2)
    L12  = L1+L2

    gQ=math.sqrt(4)
    xvec=  np.arange(-10,10.01,0.025) # 0.025
    yvec = np.arange(-10,10.01,0.025) 

    #print("epsilon", E)
    #print("and drive frequency", wd)
    #print("kappa", kappa)
    k=0
    while k < nloop :
        wl = wlist[k]    
        H = (w0-wl)*(sm.dag()*sm) + (wc-wl)*(a.dag()*a) + H1    
        LH = -complex(0,1) * (spre(H) - spost(H))
        L_final = LH + L12

        # Steady States

        rhoss = steadystate(L_final)
        rhosscav=ptrace(rhoss,0)
        rhocavsq=rhosscav*rhosscav
        k += 1

        #subplot(ceil(sqrt(nloop)), 
        #ceil(sqrt(nloop)), k)
    plt.rcParams['figure.figsize'] = (20.0, 16.0)
    fig, ax = plt.subplots()
    Q3 = qfunc(rhosscav,xvec,yvec,gQ)
    c = ax.contourf(xvec, yvec, np.real(Q3), 500, cmap=plt.cm.get_cmap('winter'))
    

#         ax.set_xlim([-3.5, 6]) # -3.5 to 6
#         ax.set_ylim([-4, 3])
    plt.colorbar(c, ax=ax)
    plt.xlabel('x')
    plt.ylabel('y')
    


        #contour(xvec,yvec,real(Q3), 500)
        #plt.plot(xvec,yvec)
        #plt.show()
        #print(rhosscav)
        #print(rhoss)
    
    coordinates = peak_local_max(Q3)    
    for coord in coordinates:
        print('peak coords:{}'.format(coord))        
        coordHeight = Q3[coord[0], coord[1]]
        print('peak height:{}'.format(coordHeight))
        plt.plot(xvec[coord[1]], yvec[coord[0]], 'o', label='{}: {}'.format(coord, coordHeight))        
    plt.legend()
    plt.show()
    #print (Q3[coordinates])
    
#     newKey = len(savedInfo.keys()) + 1
    
#     infoPacket = {}
#     infoPacket['E_i'] = E_i
#     infoPacket['wd_i'] = wd_i
#     infoPacket['coors'] = coordinates
#     infoPacket['peaks'] = Q3[coordinates]
#     infoPacket['Q3'] = Q3
#     infoPacket['xvec'] = xvec
#     infoPacket['yvec'] = yvec
#     return infoPacket

## read peaks

In [ ]:
relevantInfo = []
for peak in glob.glob(os.path.join('/mnt/c/Users/manish/Documents/GitHub/qpeak/peaks/peaksTipFiner', '*')):    
    savedInfo = pickle.load(open(peak, 'rb'))    
    
    coors = savedInfo['coors']    
#     peaks = []
#     for coor in coors:        
#         peaks.append(savedInfo['Q3'][coor[0], coor[1]])
    peaks = savedInfo['peaks']
        
    relevantInfo_i = {}
    relevantInfo_i['peakName'] = peak
    relevantInfo_i['E'] = savedInfo['E_i']
    relevantInfo_i['wd'] = savedInfo['wd_i']
    if savedInfo['wd_i'] <= 0:
        print(savedInfo['wd_i'])
        break
    
    relevantInfo_i['peaks'] = peaks
    relevantInfo_i['coors'] = coors
    relevantInfo.append(relevantInfo_i)
    print('E:{}, wd_i:{}, coords:{}, peaks:{}'.format(relevantInfo_i['E'], relevantInfo_i['wd'], relevantInfo_i['coors'],relevantInfo_i['peaks']))
    
print(len(relevantInfo))

In [ ]:
# kappa = 0.0012 # 0.0012     
# wc = 10.5665 # Cavity frequency/detuning 10.5665

# interestMinE = 0.00132
# interestMaxE = 0.001326
# interestMinwd = 10.6103
# interestMaxwd = 10.61054

# interestNoFilter = []
# for i in relevantInfo:
#     if (interestMaxE >= i['E'] and i['E'] >= interestMinE) and (interestMaxwd >= i['wd'] and i['wd'] >= interestMinwd):
#         scaledE = i['E']/kappa
#         scaledwd = (i['wd'] - wc)/kappa
#         interestNoFilter.append((i['E'], i['wd'], scaledE, scaledwd))

In [ ]:
# samples = [[i for i in relevantInfo if len(i['peaks']) == 3][10]]
# #samples = [i for i in relevantInfo if i['peakName'] == '/mnt/c/Users/manish/Documents/GitHub/qpeak/peaks/peaksOverview/1042.qpeak']

# for sample in samples:
#     routine(sample['E'], sample['wd'])    

In [ ]:
#trim similar peaks (that are within some range of each other and have peaks that are within some range)

minDist = 10 #in abs units of xvec, yvec
#minHeightDiff = 0.1 #in relevant units of Qfn. height probably would be in the range 1.0000xxxxx is permitted

peakDist= []
peakDistPost = []

packetsTrimmed = 0
peaksTrimmed = 0

for relevantInfo_i in relevantInfo:
    peaksTmp = []
    for coors, peak in zip(relevantInfo_i['coors'], relevantInfo_i['peaks']):
        peaksTmp.append((coors, peak))

    filterAgain = True    
    while filterAgain:
        filterAgain = False
        for coor1, peak1 in peaksTmp:
            for coor2, peak2 in peaksTmp:
                if not filterAgain:
                    if not (coor1[0] == coor2[0] and coor1[1] == coor2[1] and peak1 == peak2):

                        dist = np.sqrt((coor1[0] - coor2[0])**2 + (coor1[1] - coor2[1])**2)                        
                        peakDist.append(dist)
                        if dist < minDist:
                            if peak1 >= peak2:                        
                                peaksTmp = [i for i in peaksTmp if not (i[0][0] == coor2[0] and i[0][1] == coor2[1] and i[1] == peak2)]
                                peaksTrimmed += 1
                                filterAgain = True
                                #peaksTmp.remove((coor2, peak2))
                            else:
                                peaksTmp = [i for i in peaksTmp if not (i[0][0] == coor1[0] and i[0][1] == coor1[1] and i[1] == peak1)]
                                peaksTrimmed += 1
                                filterAgain = True
                                #peaksTmp.remove((coor1, peak1))                

            
    for coor1, peak1 in peaksTmp:
        for coor2, peak2 in peaksTmp:
            if coor1[0] != coor2[0] and coor1[1] != coor2[1]:
                dist = np.sqrt((coor1[0] - coor2[0])**2 + (coor1[1] - coor2[1])**2)
                peakDistPost.append(dist)
    
    newCoors = []
    newPeaks = []
    for coor, peak in peaksTmp:
        newCoors.append(coor)
        newPeaks.append(peak)
    if len(newPeaks) != len(relevantInfo_i['peaks']):
        relevantInfo_i['coors'] = newCoors
        relevantInfo_i['peaks'] = newPeaks
        packetsTrimmed += 1
    
                    
print('{} datapoints were trimmed.'.format(packetsTrimmed))    
print('{} peaks were trimmed'.format(peaksTrimmed))

plt.plot(peakDist, '.')
plt.title('distribution of distances between detected peaks pre removal')
plt.ylabel('peak distance')
plt.xlabel('arb idx')
plt.ylim(0, 50)
plt.show();

plt.plot(peakDistPost, '.')
plt.title('distribution of distances between detected peaks post removal')
plt.ylabel('peak distance')
plt.xlabel('arb idx')
plt.ylim(0, 50)
plt.show();

In [ ]:
# doublePeaks = []

# for a_i in relevantInfo:
#     if len(a_i['peaks']) == 2:
#         for b in a_i['peaks']:
#             doublePeaks.append(b)
# c = [i['peaks'][0] for i in relevantInfo if len(i['peaks']) == 1]
# concacPeaks = np.sum([c, doublePeaks])

In [ ]:
# plt.plot(concacPeaks, '.')
# plt.title('distribution of peak heights in data')
# plt.ylabel('candidate peak heights')
# plt.xlabel('arb index')
# plt.axvline(x=len(c), color='k')
# #plt.ylim(0.01,0.05)

In [ ]:
# peaksTest = []
# for relevantInfo_i in relevantInfo:
#     for j in relevantInfo_i['peaks']:
#         peaksTest.append(j)
#     plt.plot(peaksTest, '.')
# print(np.sum([j for j in peaksTest]))

In [ ]:
#todo1: play with peaktrimlimit to affect width of leaf
#todo2: if 1 peak r = nothing? that might be the grey region

numIgnored = 0
plotData = []
peakTrimLimit = 0
biPeakLimit = 0.05#0.046
siPeakLimit = 9999999
singlePeakVals = []
for relevantInfo_i in relevantInfo:
    
    trimmedPeaks = []
    foundPeaks = relevantInfo_i['peaks']
    for foundPeak_i in foundPeaks:
        #if foundPeak_i > peakTrimLimit:
        trimmedPeaks.append(foundPeak_i)
    
    if len(trimmedPeaks) == 1:
#         r = 0
#         #singlePeakVals.append(trimmedPeaks[0])
#         if trimmedPeaks[0] > siPeakLimit:
#             plotData.append((relevantInfo_i['E'], relevantInfo_i['wd'], r))
        continue
    elif len(trimmedPeaks) == 2:
        #peak0 = relevantInfo_i['peaks'][0]
        #peak1 = relevantInfo_i['peaks'][1]
        peak0 = trimmedPeaks[0]
        peak1 = trimmedPeaks[1]
        if peak0 > biPeakLimit and peak1 > biPeakLimit:
            r = 1 - (abs(peak0-peak1)/(peak0+peak1))
            plotData.append((relevantInfo_i['E'], relevantInfo_i['wd'], r))
#         if r < 0.01:
#             print(peak0)
#             print(peak1)
#             print('bbbb')
#             break
#         if relevantInfo_i['wd'] < 0:
#             print(relevantInfo_i['wd'])
#             #print('aaaa')
#             break
        
    else:
        numIgnored += 1
        #print(relevantInfo_i['E'])
        #print(relevantInfo_i['wd'])
    #ignoreif more than 2 peaks
print(len(plotData))
print(len(trimmedPeaks))
print(numIgnored)
if len(plotData) == 0:
    print('NO BISTABILITY')

In [ ]:
# interestFiltered = []
# for i in plotData:
#     if (interestMaxE >= i[0] and i[0] >= interestMinE) and (interestMaxwd >= i[1] and i[1] >= interestMinwd):
#         scaledE = i[0]/kappa
#         scaledwd = (i[1] - wc)/kappa
#         interestFiltered.append((i[0], i[1], scaledE, scaledwd))  
        
# print('Before, {} data points. After filter, {} data points.'.format(len(interestNoFilter), len(interestFiltered)))

# filteredItems = list(set(interestNoFilter) - set(interestFiltered))
# for item in filteredItems:
#     print('(E, wd): {}, {} was trimmed.'.format(item[0], item[1]))

In [ ]:
plotDataScaled = []
for i in plotData:
    item = []
    item.append(i[0]/kappa)  
    item.append((i[1] - wc)/kappa)    
    item.append(i[2])
    plotDataScaled.append(item)

In [ ]:
allDataScaled = []
for relevantInfo_i in relevantInfo:
    item = []
    item.append(relevantInfo_i['E']/kappa)
    item.append((relevantInfo_i['wd'] - wc)/kappa)
    allDataScaled.append(item)

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

saveDir = 'peaks'
plt.scatter([i[1] for i in plotDataScaled], [i[0] for i in plotDataScaled], c=[i[2] for i in plotDataScaled], s=40, cmap=plt.cm.Spectral)
plt.ylabel(r"$E/\kappa$")
plt.xlabel(r"($\mathregular{w_{drive}}- \mathregular{w_{cavity}}) / \kappa$")
#plt.ylim(min([i[0] for i in plotData]), max([i[0] for i in plotData]))

plt.ylim((min([i[0] for i in allDataScaled])), (max([i[0] for i in allDataScaled])))
plt.xlim((min([i[1] for i in allDataScaled])), (max([i[1] for i in allDataScaled])))
#plt.ylim(0.00125,0.00150)
#plt.xlim(10.6095, 10.6109)
plt.colorbar()
plt.show();

plt.scatter([i[1] for i in plotDataScaled], [i[0] for i in plotDataScaled], c=[i[2] for i in plotDataScaled], s=40, cmap=plt.cm.Spectral)
plt.ylabel(r"$E/\kappa$")
plt.xlabel(r"($\mathregular{w_{drive}}- \mathregular{w_{cavity}}) / \kappa$")
plt.ylim((min([i[0] for i in plotDataScaled])), (max([i[0] for i in plotDataScaled])))
plt.xlim((min([i[1] for i in plotDataScaled])), (max([i[1] for i in plotDataScaled])))
plt.colorbar()
plt.show();


#print("EList = np.linspace(0.0,0.004,40)")
#print("wdList = np.linspace(10.6,10.62,40)")

In [ ]:
# savedInfoTest = pickle.load(open(os.path.join(saveDir,'3.qpeak'), 'rb'))
# print(savedInfoTest.keys())
# print(savedInfoTest)

In [ ]:
# print(routine(savedInfoTest['E_i'],savedInfoTest['wd_i']))

In [ ]:
%reset